In [8]:
import sys, getopt, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import PauliFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.utils.loss_functions import L2Loss  # Qiskit ML loss, or use PyTorch's
from qiskit.quantum_info import SparsePauliOp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

root_folder = 'QNNC_hybrid'
### Globals
# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Fixed feature sizes
NUM_FEATURES = 3
NUM_QUBITS = NUM_FEATURES
NUM_TARGETS = 1

# Quantum circuit parameters
FEATURE_MAP_REPS_LIST = [1, 2, 3, 4, 5, 6]
ANSATZ_REPS_LIST = [1, 2, 3, 4, 5, 6]
ENTANGLEMENT_LIST = ['linear', 'full', 'circular']

# Training hyperparameters
LEARNING_RATE = 0.01
BATCH_SIZE = 30
NUM_EPOCHS = 100  # Adjust as needed

# K-fold cross-validation parameters
N_REPEATS = 10
TEST_SIZE = 1  # Leave-one-out cross-validation (LOOCV) is suggested since the sample size is too small

def get_qnn_torch_model(entangle, feature_map_reps, ansatz_reps):
    ### Feature Map, Ansatz, then QNN Constructor
    # a. Feature Map: Encodes NUM_FEATURES into NUM_QUBITS
    # ParameterVector for input features
    input_params = ParameterVector("x", NUM_FEATURES)

    feature_map_template = PauliFeatureMap(
        feature_dimension=NUM_FEATURES,  # This tells the template how many input parameters it structurally needs
        reps=feature_map_reps,
        entanglement=entangle
    )

    # Assign the *specific* input parameters from the vector to the template's parameter slots
    # This creates a new circuit instance containing parameters ONLY from input_params (size NUM_FEATURES)
    feature_map = feature_map_template.assign_parameters(input_params)
    print(f"Assigned feature map parameters: {feature_map.num_parameters}")

    # Create a template to find out how many parameters it needs structurally
    ansatz_template = RealAmplitudes(NUM_QUBITS, reps=ansatz_reps, entanglement=entangle)
    # ParameterVector for trainable weights - sized based on the template's structural parameters
    num_ansatz_params = ansatz_template.num_parameters
    weight_params = ParameterVector("θ", num_ansatz_params)

    # Create the ansatz circuit instance by assigning the weight parameters to the template
    ansatz = ansatz_template.assign_parameters(weight_params)
    print(f"Assigned ansatz parameters: {ansatz.num_parameters}")

    # c. Combine into a full quantum circuit
    qc = QNNCircuit(
        feature_map=feature_map,
        ansatz=ansatz_template,
    )

    # example 5.2 from Qiskit guide on binary classification
    parity = lambda x: "{:b}".format(x).count("1") % 2
    output_shape = 2  # parity = 0, 1

    sampler = Sampler()

    qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=output_shape,
        sampler=sampler,
        sparse=False,
        input_gradients=False,  # Set to True if you need gradients w.r.t. inputs
    )

    # --- 4. TorchConnector ---
    # Wrap the QNN into a PyTorch module
    initial_weights = 0.01 * (2 * np.random.rand(qnn.num_weights) - 1)
    qnn_torch_model = TorchConnector(qnn, initial_weights=torch.tensor(initial_weights, dtype=torch.float32))

    return qnn_torch_model.to(device)


class HybridModel(nn.Module):
    def __init__(self, qnn_model):
        super().__init__()
        self.qnn = qnn_model

    def forward(self, x):
        x = self.qnn(x)
        return x


def prepare_dataset_k_fold(X, y, train_indices, test_indices):
    # Separate train/test split
    X_train_raw, X_test_raw = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    X_train = X_train_raw
    X_test = X_test_raw
    full_X = np.vstack([X_train, X_test])
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(full_X)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, y_train, X_test_scaled, y_test, None, None


def get_arguments(argvs):
    _entangle = ''
    _feature_map_reps = ''
    _ansatz_reps = ''
    try:
        opts, args = getopt.getopt(argvs, "h:e:f:a:", ["entangle=", "feature_map_reps=", "ansatz_reps="])
    except getopt.GetoptError:
        print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps>')
            sys.exit()
        elif opt in ("-e", "--entangle"):
            _entangle = arg
        elif opt in ("-f", "--feature_map_reps"):
            _feature_map_reps = int(arg)
        elif opt in ("-a", "--ansatz_reps"):
            _ansatz_reps = int(arg)
    return _entangle, _feature_map_reps, _ansatz_reps


if __name__ == "__main__":
    date = '06_08_25_1'
    if not os.path.exists(f'{root_folder}/result'):
        os.makedirs(f'{root_folder}/result')
    if not os.path.exists(f'{root_folder}/logs'):
        os.makedirs(f'{root_folder}/logs')

    # Check if running in Jupyter
    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            # Running in Jupyter, use default values
            tmp1, tmp2, tmp3 = 'linear', 1, 1  # Default values
        else:
            # Running as script, parse arguments
            tmp1, tmp2, tmp3 = get_arguments(sys.argv[1:])
    except ImportError:
        # No IPython, assume script and parse arguments
        tmp1, tmp2, tmp3 = get_arguments(sys.argv[1:])

    if tmp1 != '':
        ENTANGLEMENT_LIST = [tmp1]
    if tmp2 != '':
        FEATURE_MAP_REPS_LIST = [tmp2]
    if tmp3 != '':
        ANSATZ_REPS_LIST = [tmp3]
    print(f"\nFEATURE_MAP_REPS_LIST={FEATURE_MAP_REPS_LIST} "
          f"ANSATZ_REPS_LIST={ANSATZ_REPS_LIST} ENTANGLEMENT_LIST={ENTANGLEMENT_LIST} date={date}")
    if len(FEATURE_MAP_REPS_LIST) == 1:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST[0]
    else:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST
    if len(ANSATZ_REPS_LIST) == 1:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST[0]
    else:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST
    if len(ENTANGLEMENT_LIST) == 1:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST[0]
    else:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST
    file_name = f'{root_folder}/result/FMR_{FEATURE_MAP_REPS_LIST_NAME}_AR_{ANSATZ_REPS_LIST_NAME}_E_{ENTANGLEMENT_LIST_NAME}_{date}.csv'

    print("\n--- Loading and Preprocessing Data ---")

    train_df = pd.read_csv("Training_Top3Features.csv")
    test_df = pd.read_csv("Testing_Top3Features.csv")
    X_train = train_df.drop('went_on_backorder', axis=1).values
    y_train = train_df['went_on_backorder'].values
    X_test = test_df.drop('went_on_backorder', axis=1).values
    y_test = test_df['went_on_backorder'].values

    # Combine train and test for k-fold
    X = np.vstack([X_train, X_test])
    y = np.hstack([y_train, y_test])

    print('Total number of data: ', X.shape[0])
    rkf = RepeatedKFold(n_splits=X.shape[0] // TEST_SIZE, n_repeats=N_REPEATS)

    df = pd.DataFrame(columns=['entanglement', 'feature_map_reps', 'ansatz_reps',
                               'element test', 'actual test', 'predicted test',
                               'element train', 'actual train', 'predicted train',
                               ])
    i = 0

    LOSS = nn.CrossEntropyLoss()  # use torch.long

    print("\n--- Start K-Fold Loop ---")

    for train_indices, test_indices in rkf.split(X):
        X_train_scaled, y_train, X_test_scaled, y_test, element_test, element_train = prepare_dataset_k_fold(X, y, train_indices, test_indices)

        X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
        y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
        X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
        y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)

        # Create DataLoaders
        train_dataset = TensorDataset(X_train_t, y_train_t)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        test_dataset = TensorDataset(X_test_t, y_test_t)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        print(f"Training data shape: X_train_t: {X_train_t.shape}, y_train_t: {y_train_t.shape}")
        print(f"Testing data shape: X_test_t: {X_test_t.shape}, y_test_t: {y_test_t.shape}")

        for entanglement in ENTANGLEMENT_LIST:
            for feature_map_reps in FEATURE_MAP_REPS_LIST:
                for ansatz_reps in ANSATZ_REPS_LIST:
                    # Build model
                    model = HybridModel(get_qnn_torch_model(entangle=entanglement,
                                                            feature_map_reps=feature_map_reps,
                                                            ansatz_reps=ansatz_reps)).to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

                    print(f"\n--- Starting Training {i}th---")
                    train_losses = []
                    test_losses = []

                    for epoch in range(NUM_EPOCHS):
                        # Training phase
                        model.train()
                        running_loss = 0.0
                        for batch_X, batch_y in train_loader:
                            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                            optimizer.zero_grad()  # Clear gradients
                            outputs = model(batch_X)  # Forward pass
                            loss = LOSS(outputs, batch_y)  # Calculate loss
                            loss.backward()  # Backward pass (compute gradients)
                            optimizer.step()  # Update weights
                            running_loss += loss.item() * batch_X.size(0)

                        epoch_loss = running_loss / len(train_loader.dataset)
                        train_losses.append(epoch_loss)

                        # Validation/Test phase
                        model.eval()
                        test_loss = 0.0
                        with torch.no_grad():  # Disable gradient calculations
                            for batch_X_test, batch_y_test in test_loader:
                                batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
                                outputs_test = model(batch_X_test)
                                loss_test = LOSS(outputs_test, batch_y_test)
                                test_loss += loss_test.item() * batch_X_test.size(0)

                        epoch_test_loss = test_loss / len(test_loader.dataset)
                        test_losses.append(epoch_test_loss)

                        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_loss:.4f}, Test Loss: {epoch_test_loss:.4f}")

                    print("--- Training Finished ---")

                    # --- Evaluation on Test and Training Set ---
                    model.eval()
                    all_preds = []
                    all_targets = []
                    all_preds_train = []
                    all_targets_train = []
                    with torch.no_grad():
                        for batch_X_test, batch_y_test in test_loader:
                            batch_X_test = batch_X_test.to(device)
                            outputs_test = model(batch_X_test)
                            all_preds.extend(outputs_test.cpu().numpy())
                            all_targets.extend(batch_y_test.cpu().numpy())
                        for batch_X_train, batch_y_train in train_loader:
                            batch_X_train = batch_X_train.to(device)
                            outputs_train = model(batch_X_train)
                            all_preds_train.extend(outputs_train.cpu().numpy())
                            all_targets_train.extend(batch_y_train.cpu().numpy())

                    all_preds = np.array(all_preds)
                    all_targets = np.array(all_targets)
                    all_preds_temp = []
                    for item in all_preds:
                        if item[1] > item[0]:
                            all_preds_temp.append(1)
                        else:
                            all_preds_temp.append(0)
                    all_preds = np.array(all_preds_temp)

                    all_preds_train = np.array(all_preds_train)
                    all_targets_train = np.array(all_targets_train)
                    all_preds_temp = []
                    for item in all_preds_train:
                        if item[1] > item[0]:
                            all_preds_temp.append(1)
                        else:
                            all_preds_temp.append(0)
                    all_preds_train = np.array(all_preds_temp)

                    # Calculate metrics
                    train_accuracy = accuracy_score(all_targets_train, all_preds_train)
                    test_accuracy = accuracy_score(all_targets, all_preds)
                    train_f1 = f1_score(all_targets_train, all_preds_train, zero_division=0)
                    test_f1 = f1_score(all_targets, all_preds, zero_division=0)
                    train_recall = recall_score(all_targets_train, all_preds_train, zero_division=0)
                    test_recall = recall_score(all_targets, all_preds, zero_division=0)
                    train_precision = precision_score(all_targets_train, all_preds_train, zero_division=0)
                    test_precision = precision_score(all_targets, all_preds, zero_division=0)
                    train_cm = confusion_matrix(all_targets_train, all_preds_train)
                    test_cm = confusion_matrix(all_targets, all_preds, labels=[0, 1])

                    print(f"\n--- Metrics for entanglement: {entanglement}, feature_map_reps: {feature_map_reps}, ansatz_reps: {ansatz_reps} ---")
                    print(f"Train Accuracy: {train_accuracy:.4f}")
                    print(f"Test Accuracy: {test_accuracy:.4f}")
                    print(f"Train F1 Score: {train_f1:.4f}")
                    print(f"Test F1 Score: {test_f1:.4f}")
                    print(f"Train Recall: {train_recall:.4f}")
                    print(f"Test Recall: {test_recall:.4f}")
                    print(f"Train Precision: {train_precision:.4f}")
                    print(f"Test Precision: {test_precision:.4f}")
                    print(f"Train Confusion Matrix:\n{train_cm}")
                    print(f"Test Confusion Matrix:\n{test_cm}")

                    print(torch.cuda.memory_allocated())
                    print(torch.cuda.max_memory_allocated())

                    print(f"\n--- Done for entanglement: {entanglement}, feature_map_reps: {feature_map_reps}, ansatz_reps: {ansatz_reps} ---")

                    # Add to dataframe
                    new_row = {'entanglement': entanglement,
                               'feature_map_reps': feature_map_reps,
                               'ansatz_reps': ansatz_reps,
                               'element test': element_test if element_test is not None else 'N/A',
                               'actual test': np.array(all_targets).flatten(),
                               'predicted test': np.array(all_preds).flatten(),
                               'element train': element_train if element_train is not None else 'N/A',
                               'actual train': np.array(all_targets_train).flatten(),
                               'predicted train': np.array(all_preds_train).flatten(),
                               }
                    df.loc[len(df)] = new_row
                    with np.printoptions(linewidth=10000):
                        df.to_csv(file_name, index=False)
        i += 1
    df.at[0, "info"] = [f"DATASET: Training_Top3Features.csv, Testing_Top3Features.csv, LEARNING_RATE = {LEARNING_RATE}, "
                        f"BATCH_SIZE = {BATCH_SIZE}, NUM_EPOCHS = {NUM_EPOCHS}, LOSS: {LOSS}"]
    with np.printoptions(linewidth=10000):
        df.to_csv(file_name, index=False)

Using device: cpu

FEATURE_MAP_REPS_LIST=[1] ANSATZ_REPS_LIST=[1] ENTANGLEMENT_LIST=['linear'] date=06_08_25_1

--- Loading and Preprocessing Data ---
Total number of data:  14000

--- Start K-Fold Loop ---
Training data shape: X_train_t: torch.Size([13999, 3]), y_train_t: torch.Size([13999])
Testing data shape: X_test_t: torch.Size([1, 3]), y_test_t: torch.Size([1])
Assigned feature map parameters: 3
Assigned ansatz parameters: 6

--- Starting Training 0th---


/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:89: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:91: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/100, Train Loss: 0.6693, Test Loss: 0.8009
Epoch 2/100, Train Loss: 0.6626, Test Loss: 0.7899
Epoch 3/100, Train Loss: 0.6625, Test Loss: 0.7847
Epoch 4/100, Train Loss: 0.6624, Test Loss: 0.7989
Epoch 5/100, Train Loss: 0.6625, Test Loss: 0.8171
Epoch 6/100, Train Loss: 0.6624, Test Loss: 0.8164
Epoch 7/100, Train Loss: 0.6625, Test Loss: 0.7909
Epoch 8/100, Train Loss: 0.6625, Test Loss: 0.7834
Epoch 9/100, Train Loss: 0.6624, Test Loss: 0.8061
Epoch 10/100, Train Loss: 0.6624, Test Loss: 0.8059
Epoch 11/100, Train Loss: 0.6624, Test Loss: 0.8041
Epoch 12/100, Train Loss: 0.6625, Test Loss: 0.7885
Epoch 13/100, Train Loss: 0.6625, Test Loss: 0.8006
Epoch 14/100, Train Loss: 0.6625, Test Loss: 0.7978
Epoch 15/100, Train Loss: 0.6625, Test Loss: 0.7911
Epoch 16/100, Train Loss: 0.6624, Test Loss: 0.8093
Epoch 17/100, Train Loss: 0.6626, Test Loss: 0.8029
Epoch 18/100, Train Loss: 0.6625, Test Loss: 0.7752
Epoch 19/100, Train Loss: 0.6625, Test Loss: 0.8052
Epoch 20/100, Train L

/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:89: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:91: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/100, Train Loss: 0.6416, Test Loss: 0.5388
Epoch 2/100, Train Loss: 0.6186, Test Loss: 0.5356
Epoch 3/100, Train Loss: 0.6186, Test Loss: 0.5126
Epoch 4/100, Train Loss: 0.6186, Test Loss: 0.5460
Epoch 5/100, Train Loss: 0.6186, Test Loss: 0.5193
Epoch 6/100, Train Loss: 0.6186, Test Loss: 0.5516
Epoch 7/100, Train Loss: 0.6186, Test Loss: 0.5157
Epoch 8/100, Train Loss: 0.6188, Test Loss: 0.5311
Epoch 9/100, Train Loss: 0.6186, Test Loss: 0.5437
Epoch 10/100, Train Loss: 0.6187, Test Loss: 0.5223
Epoch 11/100, Train Loss: 0.6187, Test Loss: 0.5307
Epoch 12/100, Train Loss: 0.6187, Test Loss: 0.5426
Epoch 13/100, Train Loss: 0.6185, Test Loss: 0.5092
Epoch 14/100, Train Loss: 0.6186, Test Loss: 0.5209
Epoch 15/100, Train Loss: 0.6186, Test Loss: 0.5181
Epoch 16/100, Train Loss: 0.6186, Test Loss: 0.5361
Epoch 17/100, Train Loss: 0.6187, Test Loss: 0.5216
Epoch 18/100, Train Loss: 0.6186, Test Loss: 0.5518
Epoch 19/100, Train Loss: 0.6187, Test Loss: 0.5167
Epoch 20/100, Train L

/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:89: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:91: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/100, Train Loss: 0.6408, Test Loss: 0.5438
Epoch 2/100, Train Loss: 0.6187, Test Loss: 0.5407
Epoch 3/100, Train Loss: 0.6187, Test Loss: 0.5329
Epoch 4/100, Train Loss: 0.6187, Test Loss: 0.5509
Epoch 5/100, Train Loss: 0.6185, Test Loss: 0.5737
Epoch 6/100, Train Loss: 0.6185, Test Loss: 0.5749
Epoch 7/100, Train Loss: 0.6186, Test Loss: 0.5680
Epoch 8/100, Train Loss: 0.6187, Test Loss: 0.5544
Epoch 9/100, Train Loss: 0.6187, Test Loss: 0.5697
Epoch 10/100, Train Loss: 0.6186, Test Loss: 0.5629
Epoch 11/100, Train Loss: 0.6186, Test Loss: 0.5746
Epoch 12/100, Train Loss: 0.6187, Test Loss: 0.5607
Epoch 13/100, Train Loss: 0.6186, Test Loss: 0.5426
Epoch 14/100, Train Loss: 0.6186, Test Loss: 0.5625
Epoch 15/100, Train Loss: 0.6186, Test Loss: 0.5401
Epoch 16/100, Train Loss: 0.6186, Test Loss: 0.5461
Epoch 17/100, Train Loss: 0.6187, Test Loss: 0.5370
Epoch 18/100, Train Loss: 0.6186, Test Loss: 0.5677
Epoch 19/100, Train Loss: 0.6186, Test Loss: 0.5512
Epoch 20/100, Train L

/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:89: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1205221547.py:91: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/100, Train Loss: 0.6696, Test Loss: 0.4673
Epoch 2/100, Train Loss: 0.6625, Test Loss: 0.4544
Epoch 3/100, Train Loss: 0.6625, Test Loss: 0.4582
Epoch 4/100, Train Loss: 0.6625, Test Loss: 0.4504
Epoch 5/100, Train Loss: 0.6624, Test Loss: 0.4539
Epoch 6/100, Train Loss: 0.6626, Test Loss: 0.4459
Epoch 7/100, Train Loss: 0.6626, Test Loss: 0.4600
Epoch 8/100, Train Loss: 0.6625, Test Loss: 0.4530
Epoch 9/100, Train Loss: 0.6626, Test Loss: 0.4619
Epoch 10/100, Train Loss: 0.6626, Test Loss: 0.4464
Epoch 11/100, Train Loss: 0.6625, Test Loss: 0.4781
Epoch 12/100, Train Loss: 0.6625, Test Loss: 0.4499
Epoch 13/100, Train Loss: 0.6625, Test Loss: 0.4387
Epoch 14/100, Train Loss: 0.6625, Test Loss: 0.4566


KeyboardInterrupt: 

In [9]:
import sys, getopt, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import PauliFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.utils.loss_functions import L2Loss
from qiskit.quantum_info import SparsePauliOp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

root_folder = 'QNNC_hybrid'
### Globals
# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Fixed feature sizes
NUM_FEATURES = 3
NUM_QUBITS = NUM_FEATURES
NUM_TARGETS = 1

# Quantum circuit parameters
FEATURE_MAP_REPS_LIST = [1, 2, 3, 4, 5, 6]
ANSATZ_REPS_LIST = [1, 2, 3, 4, 5, 6]
ENTANGLEMENT_LIST = ['linear', 'full', 'circular']

# Training hyperparameters
LEARNING_RATE = 0.01
BATCH_SIZE = 30
NUM_EPOCHS = 100

# Training repetition parameters
N_REPEATS = 1

def get_qnn_torch_model(entangle, feature_map_reps, ansatz_reps):
    input_params = ParameterVector("x", NUM_FEATURES)
    feature_map_template = PauliFeatureMap(
        feature_dimension=NUM_FEATURES,
        reps=feature_map_reps,
        entanglement=entangle
    )
    feature_map = feature_map_template.assign_parameters(input_params)
    print(f"Assigned feature map parameters: {feature_map.num_parameters}")
    ansatz_template = RealAmplitudes(NUM_QUBITS, reps=ansatz_reps, entanglement=entangle)
    num_ansatz_params = ansatz_template.num_parameters
    weight_params = ParameterVector("θ", num_ansatz_params)
    ansatz = ansatz_template.assign_parameters(weight_params)
    print(f"Assigned ansatz parameters: {ansatz.num_parameters}")
    qc = QNNCircuit(
        feature_map=feature_map,
        ansatz=ansatz_template,
    )
    parity = lambda x: "{:b}".format(x).count("1") % 2
    output_shape = 2
    sampler = Sampler()
    qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=output_shape,
        sampler=sampler,
        sparse=False,
        input_gradients=False,
    )
    initial_weights = 0.01 * (2 * np.random.rand(qnn.num_weights) - 1)
    qnn_torch_model = TorchConnector(qnn, initial_weights=torch.tensor(initial_weights, dtype=torch.float32))
    return qnn_torch_model.to(device)

class HybridModel(nn.Module):
    def __init__(self, qnn_model):
        super().__init__()
        self.qnn = qnn_model
    def forward(self, x):
        x = self.qnn(x)
        return x

def prepare_dataset_k_fold(X_train, y_train, X_test, y_test):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(np.vstack([X_train, X_test]))
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, y_train, X_test_scaled, y_test, None, None

def get_arguments(argvs):
    _entangle = ''
    _feature_map_reps = ''
    _ansatz_reps = ''
    try:
        opts, args = getopt.getopt(argvs, "h:e:f:a:", ["entangle=", "feature_map_reps=", "ansatz_reps="])
    except getopt.GetoptError:
        print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps>')
            sys.exit()
        elif opt in ("-e", "--entangle"):
            _entangle = arg
        elif opt in ("-f", "--feature_map_reps"):
            _feature_map_reps = int(arg)
        elif opt in ("-a", "--ansatz_reps"):
            _ansatz_reps = int(arg)
    return _entangle, _feature_map_reps, _ansatz_reps

if __name__ == "__main__":
    date = '06_08_25_1'
    if not os.path.exists(f'{root_folder}/result'):
        os.makedirs(f'{root_folder}/result')
    if not os.path.exists(f'{root_folder}/logs'):
        os.makedirs(f'{root_folder}/logs')
    if not os.path.exists(f'{root_folder}/figures'):
        os.makedirs(f'{root_folder}/figures')

    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            tmp1, tmp2, tmp3 = 'linear', 1, 1
        else:
            tmp1, tmp2, tmp3 = get_arguments(sys.argv[1:])
    except ImportError:
        tmp1, tmp2, tmp3 = get_arguments(sys.argv[1:])

    if tmp1 != '':
        ENTANGLEMENT_LIST = [tmp1]
    if tmp2 != '':
        FEATURE_MAP_REPS_LIST = [tmp2]
    if tmp3 != '':
        ANSATZ_REPS_LIST = [tmp3]
    print(f"\nFEATURE_MAP_REPS_LIST={FEATURE_MAP_REPS_LIST} "
          f"ANSATZ_REPS_LIST={ANSATZ_REPS_LIST} ENTANGLEMENT_LIST={ENTANGLEMENT_LIST} date={date}")
    if len(FEATURE_MAP_REPS_LIST) == 1:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST[0]
    else:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST
    if len(ANSATZ_REPS_LIST) == 1:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST[0]
    else:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST
    if len(ENTANGLEMENT_LIST) == 1:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST[0]
    else:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST
    file_name = f'{root_folder}/result/FMR_{FEATURE_MAP_REPS_LIST_NAME}_AR_{ANSATZ_REPS_LIST_NAME}_E_{ENTANGLEMENT_LIST_NAME}_{date}.csv'

    print("\n--- Loading and Preprocessing Data ---")

    train_df = pd.read_csv("Training_Top3Features.csv")
    test_df = pd.read_csv("Testing_Top3Features.csv")
    X_train = train_df.drop('went_on_backorder', axis=1).values
    y_train = train_df['went_on_backorder'].values
    X_test = test_df.drop('went_on_backorder', axis=1).values
    y_test = test_df['went_on_backorder'].values

    print('Training data size: ', X_train.shape[0])
    print('Testing data size: ', X_test.shape[0])

    df = pd.DataFrame(columns=['entanglement', 'feature_map_reps', 'ansatz_reps',
                               'element test', 'actual test', 'predicted test',
                               'element train', 'actual train', 'predicted train'])

    LOSS = nn.CrossEntropyLoss()

    print("\n--- Start Training Loop ---")

    for i in range(N_REPEATS):
        X_train_scaled, y_train, X_test_scaled, y_test, element_test, element_train = prepare_dataset_k_fold(X_train, y_train, X_test, y_test)

        X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
        y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
        X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
        y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)

        train_dataset = TensorDataset(X_train_t, y_train_t)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        test_dataset = TensorDataset(X_test_t, y_test_t)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        print(f"Training data shape: X_train_t: {X_train_t.shape}, y_train_t: {y_train_t.shape}")
        print(f"Testing data shape: X_test_t: {X_test_t.shape}, y_test_t: {y_test_t.shape}")

        for entanglement in ENTANGLEMENT_LIST:
            for feature_map_reps in FEATURE_MAP_REPS_LIST:
                for ansatz_reps in ANSATZ_REPS_LIST:
                    model = HybridModel(get_qnn_torch_model(entangle=entanglement,
                                                            feature_map_reps=feature_map_reps,
                                                            ansatz_reps=ansatz_reps)).to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

                    print(f"\n--- Starting Training {i+1}th ---")
                    train_losses = []
                    test_losses = []

                    for epoch in range(NUM_EPOCHS):
                        model.train()
                        running_loss = 0.0
                        for batch_X, batch_y in train_loader:
                            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                            optimizer.zero_grad()
                            outputs = model(batch_X)
                            loss = LOSS(outputs, batch_y)
                            loss.backward()
                            optimizer.step()
                            running_loss += loss.item() * batch_X.size(0)

                        epoch_loss = running_loss / len(train_loader.dataset)
                        train_losses.append(epoch_loss)

                        model.eval()
                        test_loss = 0.0
                        with torch.no_grad():
                            for batch_X_test, batch_y_test in test_loader:
                                batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
                                outputs_test = model(batch_X_test)
                                loss_test = LOSS(outputs_test, batch_y_test)
                                test_loss += loss_test.item() * batch_X_test.size(0)

                        epoch_test_loss = test_loss / len(test_loader.dataset)
                        test_losses.append(epoch_test_loss)

                        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_loss:.4f}, Test Loss: {epoch_test_loss:.4f}")

                    print("--- Training Finished ---")

                    model.eval()
                    all_preds = []
                    all_targets = []
                    all_preds_train = []
                    all_targets_train = []
                    with torch.no_grad():
                        for batch_X_test, batch_y_test in test_loader:
                            batch_X_test = batch_X_test.to(device)
                            outputs_test = model(batch_X_test)
                            all_preds.extend(outputs_test.cpu().numpy())
                            all_targets.extend(batch_y_test.cpu().numpy())
                        for batch_X_train, batch_y_train in train_loader:
                            batch_X_train = batch_X_train.to(device)
                            outputs_train = model(batch_X_train)
                            all_preds_train.extend(outputs_train.cpu().numpy())
                            all_targets_train.extend(batch_y_train.cpu().numpy())

                    all_preds = np.array(all_preds)
                    all_targets = np.array(all_targets)
                    all_preds_temp = []
                    for item in all_preds:
                        if item[1] > item[0]:
                            all_preds_temp.append(1)
                        else:
                            all_preds_temp.append(0)
                    all_preds = np.array(all_preds_temp)

                    all_preds_train = np.array(all_preds_train)
                    all_targets_train = np.array(all_targets_train)
                    all_preds_temp = []
                    for item in all_preds_train:
                        if item[1] > item[0]:
                            all_preds_temp.append(1)
                        else:
                            all_preds_temp.append(0)
                    all_preds_train = np.array(all_preds_temp)

                    train_accuracy = accuracy_score(all_targets_train, all_preds_train)
                    test_accuracy = accuracy_score(all_targets, all_preds)
                    train_f1 = f1_score(all_targets_train, all_preds_train, zero_division=0)
                    test_f1 = f1_score(all_targets, all_preds, zero_division=0)
                    train_recall = recall_score(all_targets_train, all_preds_train, zero_division=0)
                    test_recall = recall_score(all_targets, all_preds, zero_division=0)
                    train_precision = precision_score(all_targets_train, all_preds_train, zero_division=0)
                    test_precision = precision_score(all_targets, all_preds, zero_division=0)
                    train_cm = confusion_matrix(all_targets_train, all_preds_train)
                    test_cm = confusion_matrix(all_targets, all_preds, labels=[0, 1])

                    print(f"\n--- Metrics for entanglement: {entanglement}, feature_map_reps: {feature_map_reps}, ansatz_reps: {ansatz_reps} ---")
                    print(f"Train Accuracy: {train_accuracy:.4f}")
                    print(f"Test Accuracy: {test_accuracy:.4f}")
                    print(f"Train F1 Score: {train_f1:.4f}")
                    print(f"Test F1 Score: {test_f1:.4f}")
                    print(f"Train Recall: {train_recall:.4f}")
                    print(f"Test Recall: {test_recall:.4f}")
                    print(f"Train Precision: {train_precision:.4f}")
                    print(f"Test Precision: {test_precision:.4f}")
                    print(f"Train Confusion Matrix:\n{train_cm}")
                    print(f"Test Confusion Matrix:\n{test_cm}")

                    plt.figure(figsize=(8, 6))
                    sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
                    plt.title(f'Train Confusion Matrix\nEntanglement: {entanglement}, FMR: {feature_map_reps}, AR: {ansatz_reps}')
                    plt.xlabel('Predicted')
                    plt.ylabel('Actual')
                    plt.savefig(f'{root_folder}/figures/train_cm_{entanglement}_fmr{feature_map_reps}_ar{ansatz_reps}_rep{i+1}_{date}.png')
                    plt.close()

                    plt.figure(figsize=(8, 6))
                    sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
                    plt.title(f'Test Confusion Matrix\nEntanglement: {entanglement}, FMR: {feature_map_reps}, AR: {ansatz_reps}')
                    plt.xlabel('Predicted')
                    plt.ylabel('Actual')
                    plt.savefig(f'{root_folder}/figures/test_cm_{entanglement}_fmr{feature_map_reps}_ar{ansatz_reps}_rep{i+1}_{date}.png')
                    plt.close()

                    print(torch.cuda.memory_allocated())
                    print(torch.cuda.max_memory_allocated())

                    print(f"\n--- Done for entanglement: {entanglement}, feature_map_reps: {feature_map_reps}, ansatz_reps: {ansatz_reps} ---")

                    new_row = {'entanglement': entanglement,
                               'feature_map_reps': feature_map_reps,
                               'ansatz_reps': ansatz_reps,
                               'element test': element_test if element_test is not None else 'N/A',
                               'actual test': np.array(all_targets).flatten(),
                               'predicted test': np.array(all_preds).flatten(),
                               'element train': element_train if element_train is not None else 'N/A',
                               'actual train': np.array(all_targets_train).flatten(),
                               'predicted train': np.array(all_preds_train).flatten()}
                    df.loc[len(df)] = new_row
                    with np.printoptions(linewidth=10000):
                        df.to_csv(file_name, index=False)

    df.at[0, "info"] = [f"DATASET: Training_Top3Features.csv, Testing_Top3Features.csv, LEARNING_RATE = {LEARNING_RATE}, "
                        f"BATCH_SIZE = {BATCH_SIZE}, NUM_EPOCHS = {NUM_EPOCHS}, LOSS: {LOSS}"]
    with np.printoptions(linewidth=10000):
        df.to_csv(file_name, index=False)

Using device: cpu

FEATURE_MAP_REPS_LIST=[1] ANSATZ_REPS_LIST=[1] ENTANGLEMENT_LIST=['linear'] date=06_08_25_1

--- Loading and Preprocessing Data ---
Training data size:  10000
Testing data size:  4000

--- Start Training Loop ---
Training data shape: X_train_t: torch.Size([10000, 3]), y_train_t: torch.Size([10000])
Testing data shape: X_test_t: torch.Size([4000, 3]), y_test_t: torch.Size([4000])
Assigned feature map parameters: 3
Assigned ansatz parameters: 6

--- Starting Training 1th ---


/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/713294462.py:70: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/713294462.py:71: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or s

Epoch 1/100, Train Loss: 0.6687, Test Loss: 0.6664
Epoch 2/100, Train Loss: 0.6613, Test Loss: 0.6651
Epoch 3/100, Train Loss: 0.6615, Test Loss: 0.6652
Epoch 4/100, Train Loss: 0.6617, Test Loss: 0.6652
Epoch 5/100, Train Loss: 0.6614, Test Loss: 0.6650
Epoch 6/100, Train Loss: 0.6614, Test Loss: 0.6651
Epoch 7/100, Train Loss: 0.6614, Test Loss: 0.6659
Epoch 8/100, Train Loss: 0.6614, Test Loss: 0.6656
Epoch 9/100, Train Loss: 0.6615, Test Loss: 0.6657
Epoch 10/100, Train Loss: 0.6613, Test Loss: 0.6654
Epoch 11/100, Train Loss: 0.6613, Test Loss: 0.6654
Epoch 12/100, Train Loss: 0.6614, Test Loss: 0.6653
Epoch 13/100, Train Loss: 0.6615, Test Loss: 0.6651
Epoch 14/100, Train Loss: 0.6612, Test Loss: 0.6650
Epoch 15/100, Train Loss: 0.6614, Test Loss: 0.6649
Epoch 16/100, Train Loss: 0.6614, Test Loss: 0.6659
Epoch 17/100, Train Loss: 0.6614, Test Loss: 0.6656
Epoch 18/100, Train Loss: 0.6612, Test Loss: 0.6660
Epoch 19/100, Train Loss: 0.6615, Test Loss: 0.6661
Epoch 20/100, Train L

In [10]:
import sys, getopt, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import PauliFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.circuit.library import QNNCircuit

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

root_folder = 'QNNC_hybrid'
torch.manual_seed(42)
np.random.seed(42)

# ✅ CHANGE: Use 3 input features (matching CSVs)
NUM_FEATURES = 3
NUM_QUBITS = NUM_FEATURES
NUM_TARGETS = 1

# ✅ Keep only 1 value for reps (you wanted 3 training sessions only)
FEATURE_MAP_REPS_LIST = [1]
ANSATZ_REPS_LIST = [1]
ENTANGLEMENT_LIST = ['linear', 'full', 'circular']

# ✅ Tune for better speed
LEARNING_RATE = 0.01
BATCH_SIZE = 256
NUM_EPOCHS = 30

def get_qnn_torch_model(entangle, feature_map_reps, ansatz_reps):
    input_params = ParameterVector("x", NUM_FEATURES)
    feature_map_template = PauliFeatureMap(feature_dimension=NUM_FEATURES, reps=feature_map_reps, entanglement=entangle)
    feature_map = feature_map_template.assign_parameters(input_params)

    ansatz_template = RealAmplitudes(NUM_QUBITS, reps=ansatz_reps, entanglement=entangle)
    num_ansatz_params = ansatz_template.num_parameters
    weight_params = ParameterVector("θ", num_ansatz_params)
    ansatz = ansatz_template.assign_parameters(weight_params)

    qc = QNNCircuit(feature_map=feature_map, ansatz=ansatz_template)
    parity = lambda x: "{:b}".format(x).count("1") % 2
    output_shape = 2

    sampler = Sampler()
    qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=output_shape,
        sampler=sampler,
        sparse=False,
        input_gradients=False,
    )

    initial_weights = 0.01 * (2 * np.random.rand(qnn.num_weights) - 1)
    qnn_torch_model = TorchConnector(qnn, initial_weights=torch.tensor(initial_weights, dtype=torch.float32))
    return qnn_torch_model.to(device)

class HybridModel(nn.Module):
    def __init__(self, qnn_model):
        super().__init__()
        self.qnn = qnn_model

    def forward(self, x):
        return self.qnn(x)

if __name__ == "__main__":
    date = '06_08_25_1'
    os.makedirs(f'{root_folder}/result', exist_ok=True)
    os.makedirs(f'{root_folder}/logs', exist_ok=True)

    file_name = f'{root_folder}/result/FMR_1_AR_1_E_ALL_{date}.csv'

    print("\n--- Loading and Preprocessing Data ---")

    train_df = pd.read_csv("Training_Top3Features.csv")
    test_df = pd.read_csv("Testing_Top3Features.csv")
    X_train_raw = train_df.drop('went_on_backorder', axis=1).values
    y_train = train_df['went_on_backorder'].values
    X_test_raw = test_df.drop('went_on_backorder', axis=1).values
    y_test = test_df['went_on_backorder'].values

    print(f"Train: {X_train_raw.shape}, Test: {X_test_raw.shape}")

    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(np.vstack([X_train_raw, X_test_raw]))
    X_train = scaler.transform(X_train_raw)
    X_test = scaler.transform(X_test_raw)

    X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)

    train_dataset = TensorDataset(X_train_t, y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(X_test_t, y_test_t)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    df = pd.DataFrame(columns=['entanglement', 'feature_map_reps', 'ansatz_reps',
                               'train_accuracy', 'test_accuracy',
                               'train_f1', 'test_f1',
                               'train_recall', 'test_recall',
                               'train_precision', 'test_precision'])

    LOSS = nn.CrossEntropyLoss()

    for entanglement in ENTANGLEMENT_LIST:
        print(f"\n--- Training with entanglement = {entanglement} ---")

        model = HybridModel(get_qnn_torch_model(entangle=entanglement, feature_map_reps=1, ansatz_reps=1)).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(NUM_EPOCHS):
            model.train()
            total_loss = 0
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = LOSS(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * batch_X.size(0)

            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {total_loss / len(train_loader.dataset):.4f}")

        print("--- Training Complete ---")
        model.eval()
        def predict(loader):
            all_preds, all_true = [], []
            with torch.no_grad():
                for batch_X, batch_y in loader:
                    outputs = model(batch_X.to(device)).cpu().numpy()
                    all_preds.extend(outputs)
                    all_true.extend(batch_y.cpu().numpy())
            pred_classes = [1 if o[1] > o[0] else 0 for o in all_preds]
            return np.array(all_true), np.array(pred_classes)

        y_true_train, y_pred_train = predict(train_loader)
        y_true_test, y_pred_test = predict(test_loader)

        # Metrics
        metrics = {
            'entanglement': entanglement,
            'feature_map_reps': 1,
            'ansatz_reps': 1,
            'train_accuracy': accuracy_score(y_true_train, y_pred_train),
            'test_accuracy': accuracy_score(y_true_test, y_pred_test),
            'train_f1': f1_score(y_true_train, y_pred_train),
            'test_f1': f1_score(y_true_test, y_pred_test),
            'train_recall': recall_score(y_true_train, y_pred_train),
            'test_recall': recall_score(y_true_test, y_pred_test),
            'train_precision': precision_score(y_true_train, y_pred_train),
            'test_precision': precision_score(y_true_test, y_pred_test)
        }
        print(f"\n--- Metrics for entanglement = {entanglement} ---")
        for k, v in metrics.items():
            if isinstance(v, float):
                print(f"{k}: {v:.4f}")
            else:
                print(f"{k}: {v}")
        print("Train Confusion Matrix:\n", confusion_matrix(y_true_train, y_pred_train))
        print("Test Confusion Matrix:\n", confusion_matrix(y_true_test, y_pred_test))

        df.loc[len(df)] = metrics
        df.to_csv(file_name, index=False)


Using device: cpu

--- Loading and Preprocessing Data ---
Train: (10000, 3), Test: (4000, 3)

--- Training with entanglement = linear ---


/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:57: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:58: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/30, Loss: 0.6849
Epoch 2/30, Loss: 0.6728
Epoch 3/30, Loss: 0.6671
Epoch 4/30, Loss: 0.6641
Epoch 5/30, Loss: 0.6618
Epoch 6/30, Loss: 0.6611
Epoch 7/30, Loss: 0.6611
Epoch 8/30, Loss: 0.6611
Epoch 9/30, Loss: 0.6610
Epoch 10/30, Loss: 0.6611
Epoch 11/30, Loss: 0.6610
Epoch 12/30, Loss: 0.6611
Epoch 13/30, Loss: 0.6611
Epoch 14/30, Loss: 0.6611
Epoch 15/30, Loss: 0.6610
Epoch 16/30, Loss: 0.6611
Epoch 17/30, Loss: 0.6610
Epoch 18/30, Loss: 0.6610
Epoch 19/30, Loss: 0.6610
Epoch 20/30, Loss: 0.6610
Epoch 21/30, Loss: 0.6610
Epoch 22/30, Loss: 0.6611
Epoch 23/30, Loss: 0.6611
Epoch 24/30, Loss: 0.6611
Epoch 25/30, Loss: 0.6610
Epoch 26/30, Loss: 0.6610
Epoch 27/30, Loss: 0.6610
Epoch 28/30, Loss: 0.6610
Epoch 29/30, Loss: 0.6610
Epoch 30/30, Loss: 0.6610
--- Training Complete ---

--- Metrics for entanglement = linear ---
entanglement: linear
feature_map_reps: 1
ansatz_reps: 1
train_accuracy: 0.5835
test_accuracy: 0.5695
train_f1: 0.6415
test_f1: 0.6192
train_recall: 0.7454
test_

/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:57: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:58: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/30, Loss: 0.6696
Epoch 2/30, Loss: 0.6318
Epoch 3/30, Loss: 0.6168
Epoch 4/30, Loss: 0.6110
Epoch 5/30, Loss: 0.6095
Epoch 6/30, Loss: 0.6093
Epoch 7/30, Loss: 0.6094
Epoch 8/30, Loss: 0.6093
Epoch 9/30, Loss: 0.6094
Epoch 10/30, Loss: 0.6093
Epoch 11/30, Loss: 0.6093
Epoch 12/30, Loss: 0.6093
Epoch 13/30, Loss: 0.6093
Epoch 14/30, Loss: 0.6094
Epoch 15/30, Loss: 0.6093
Epoch 16/30, Loss: 0.6093
Epoch 17/30, Loss: 0.6093
Epoch 18/30, Loss: 0.6093
Epoch 19/30, Loss: 0.6093
Epoch 20/30, Loss: 0.6093
Epoch 21/30, Loss: 0.6093
Epoch 22/30, Loss: 0.6093
Epoch 23/30, Loss: 0.6093
Epoch 24/30, Loss: 0.6092
Epoch 25/30, Loss: 0.6092
Epoch 26/30, Loss: 0.6093
Epoch 27/30, Loss: 0.6093
Epoch 28/30, Loss: 0.6093
Epoch 29/30, Loss: 0.6093
Epoch 30/30, Loss: 0.6093
--- Training Complete ---

--- Metrics for entanglement = full ---
entanglement: full
feature_map_reps: 1
ansatz_reps: 1
train_accuracy: 0.8137
test_accuracy: 0.7762
train_f1: 0.8027
test_f1: 0.7578
train_recall: 0.7578
test_reca

/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:57: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
/var/folders/ly/dps_wwpj7972w23hbq80tr9r0000gn/T/ipykernel_41057/1950909632.py:58: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(
/Users/pawanpahune/New_Research_Deakin/venv/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or

Epoch 1/30, Loss: 0.6750
Epoch 2/30, Loss: 0.6483
Epoch 3/30, Loss: 0.6328
Epoch 4/30, Loss: 0.6243
Epoch 5/30, Loss: 0.6198
Epoch 6/30, Loss: 0.6177
Epoch 7/30, Loss: 0.6169
Epoch 8/30, Loss: 0.6166
Epoch 9/30, Loss: 0.6165
Epoch 10/30, Loss: 0.6164
Epoch 11/30, Loss: 0.6165
Epoch 12/30, Loss: 0.6165
Epoch 13/30, Loss: 0.6165
Epoch 14/30, Loss: 0.6164
Epoch 15/30, Loss: 0.6164
Epoch 16/30, Loss: 0.6165
Epoch 17/30, Loss: 0.6165
Epoch 18/30, Loss: 0.6164
Epoch 19/30, Loss: 0.6164
Epoch 20/30, Loss: 0.6165
Epoch 21/30, Loss: 0.6165
Epoch 22/30, Loss: 0.6165
Epoch 23/30, Loss: 0.6165
Epoch 24/30, Loss: 0.6165
Epoch 25/30, Loss: 0.6165
Epoch 26/30, Loss: 0.6164
Epoch 27/30, Loss: 0.6165
Epoch 28/30, Loss: 0.6165
Epoch 29/30, Loss: 0.6165
Epoch 30/30, Loss: 0.6164
--- Training Complete ---

--- Metrics for entanglement = circular ---
entanglement: circular
feature_map_reps: 1
ansatz_reps: 1
train_accuracy: 0.7300
test_accuracy: 0.6845
train_f1: 0.6964
test_f1: 0.6357
train_recall: 0.6192
t